### Notebook for the exploratory analysis of hepatocytes multiome data with `MultiVI`

- **Developed by**: Carlos Talavera-López
- **Institute of Computational Biology - Computational Health Department - Helmholtz Munich**
- v220829 

### Import required packages 

In [1]:
import scvi
import anndata
import numpy as np
import scanpy as sc

Global seed set to 0
/Users/cartalop/mambaforge/envs/multiVI/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/cartalop/mambaforge/envs/multiVI/lib/python3.9/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/Users/cartalop/mambaforge/envs/multiVI/lib/python3.9/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/Users/cartalop/mambaforge/envs/multiVI/lib/python3.9/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util

### Set up working environment

In [2]:
scvi.settings.seed = 1712
np.random.seed(1769)

sc.settings.verbosity = 3
sc.logging.print_versions()
sc.settings.set_figure_params(dpi = 180, color_map = 'magma_r', dpi_save = 300, vector_friendly = True, format = 'svg')

Global seed set to 1712


-----
anndata     0.9.1
scanpy      1.9.3
-----
PIL                 9.5.0
absl                NA
appnope             0.1.3
asttokens           NA
backcall            0.2.0
brotli              NA
certifi             2022.12.07
cffi                1.15.1
charset_normalizer  3.1.0
chex                0.1.7
colorama            0.4.6
comm                0.1.3
contextlib2         NA
cycler              0.10.0
cython_runtime      NA
dateutil            2.8.2
debugpy             1.6.7
decorator           5.1.1
docrep              0.3.2
executing           1.2.0
flax                0.6.1
fsspec              2023.4.0
gmpy2               2.1.2
h5py                3.8.0
idna                3.4
igraph              0.10.4
importlib_resources NA
ipykernel           6.22.0
jax                 0.4.8
jaxlib              0.4.7
jedi                0.18.2
joblib              1.2.0
kiwisolver          1.4.4
leidenalg           0.9.1
lightning_fabric    1.9.4
lightning_utilities 0.8.0
llvmlite            0.3

In [3]:
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}
%config InlineBackend.figure_format='retina'

### Read in 10X multiome samples

In [4]:
adata_1 = sc.read_10x_h5("../MDA_hepatocytes/data/MO12_sample_files/MO12_filtered_feature_bc_matrix.h5")
adata_1.var_names_make_unique()
adata_1

reading ../MDA_hepatocytes/data/MO12_sample_files/MO12_filtered_feature_bc_matrix.h5
 (0:00:01)


AnnData object with n_obs × n_vars = 4581 × 22441
    var: 'gene_ids', 'feature_types', 'genome'

In [8]:
adata_1.var['feature_types'].value_counts()

feature_types
Gene Expression    22441
Name: count, dtype: int64

In [5]:
adata_2 = sc.read_10x_h5("../MDA_hepatocytes/data/MO9_sample_files/MO9_filtered_feature_bc_matrix.h5")
adata_2.var_names_make_unique()
adata_2

reading ../MDA_hepatocytes/data/MO9_sample_files/MO9_filtered_feature_bc_matrix.h5
 (0:00:02)


AnnData object with n_obs × n_vars = 7421 × 22441
    var: 'gene_ids', 'feature_types', 'genome'

In [9]:
adata_2.var['feature_types'].value_counts()

feature_types
Gene Expression    22441
Name: count, dtype: int64

In [6]:
adata_3 = sc.read_10x_h5("../MDA_hepatocytes/data/MO6_sample_files/MO6_filtered_feature_bc_matrix.h5")
adata_3.var_names_make_unique()
adata_3

reading ../MDA_hepatocytes/data/MO6_sample_files/MO6_filtered_feature_bc_matrix.h5
 (0:00:03)


AnnData object with n_obs × n_vars = 5020 × 22441
    var: 'gene_ids', 'feature_types', 'genome'

In [10]:
adata_3.var['feature_types'].value_counts()

feature_types
Gene Expression    22441
Name: count, dtype: int64

### Concatenate samples into a single object

In [11]:
adata = adata_1.concatenate(adata_2, adata_3, batch_key = 'sample', batch_categories = ['M012', 'M09', 'M06'], join = 'inner')
adata

/Users/cartalop/mambaforge/envs/multiVI/lib/python3.9/site-packages/anndata/_core/anndata.py:1755: FutureWarning: The AnnData.concatenate method is deprecated in favour of the anndata.concat function. Please use anndata.concat instead.

See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  warnings.warn(


AnnData object with n_obs × n_vars = 17022 × 22441
    obs: 'sample'
    var: 'gene_ids', 'feature_types', 'genome'

### Split to three datasets by modality (RNA, ATAC, Multiome)

- Here we also add a composite to train the model with single-modalities. To assign a vlaue to `n` you need to divide the number of cells for one of the modalities into the total number of samples. 
- In this case we have three samples and a total number of cells of 17022. So we will use `n = 5674`. This value will change depending on the number of samples analysed and the total number of cells after concatenation. 

In [ ]:
n = 5674

In [ ]:
adata_rna = adata[:n, adata.var.modality == "Gene Expression"].copy()
adata_paired = adata[n:2 * n].copy()
adata_atac = adata[2 * n:, adata.var.modality == "Peaks"].copy()

In [ ]:
adata_rna

In [ ]:
adata_atac

### Concatenate data subsets

In [ ]:
adata_mvi = scvi.data.organize_multiome_anndatas(adata_paired, adata_rna, adata_atac)
adata_mvi

In [ ]:
adata_mvi.obs

### `MultiVI` requires the genomic features to be first in the `anndata` object. We arrange for this below.

In [ ]:
adata_mvi = adata_mvi[:, adata_mvi.var["modality"].argsort()].copy()
adata_mvi.var

### Remove potentially empty cells

In [ ]:
print(adata_mvi.shape)
sc.pp.filter_genes(adata_mvi, min_cells = int(adata_mvi.shape[0] * 0.01))
print(adata_mvi.shape)

### Set up model

In [ ]:
scvi.model.MULTIVI.setup_anndata(adata_mvi, batch_key = 'modality')

In [ ]:
mvi = scvi.model.MULTIVI(
    adata_mvi,
    n_genes = (adata_mvi.var['modality']=='Gene Expression').sum(),
    n_regions = (adata_mvi.var['modality']=='Peaks').sum(),
)
mvi.view_anndata_setup()

### Train model

In [ ]:
mvi.train()

### Save and load `MultiVI` model

In [ ]:
mvi.save("trained_multivi")
mvi = scvi.model.MULTIVI.load("trained_multivi", adata = adata_mvi)

### Visualise latent space 

In [ ]:
adata_mvi.obsm["MultiVI_latent"] = mvi.get_latent_representation()
sc.pp.neighbors(adata_mvi, use_rep = "MultiVI_latent")
sc.tl.umap(adata_mvi, min_dist = 0.3, spread = 1)
sc.tl.leiden(adata_mvi, resolution = 1)
sc.pl.umap(adata_mvi, color = ['modality', 'leiden'], frameon = False, size = 1)

### Impute gene values for joint annotation

In a well-mixed space, `MultiVI` can seamlessly impute the missing modalities for single-modality cells.

In [ ]:
imputed_expression = mvi.get_normalized_expression()

In [ ]:
gene_idx = np.where(adata_mvi.var.index == "CD3G")[0]
adata_mvi.obs["CD3G_imputed"] = imputed_expression.iloc[:, gene_idx]
sc.pl.umap(adata_mvi, color='CD3G_imputed')